## Welcome to your notebook for a Group Challenge.


#### Load the necessary packages

note that this notebook can only run in the local arcgis pro, and is not able to run in the arcgis online because arcgis online does not support arcpy package

In [1]:
from arcgis.gis import GIS
gis = GIS("home")
import os
import glob
import zipfile
import arcpy
import pandas as pd
arcpy.CheckOutExtension("Spatial")

/opt/conda/lib/python3.6/site-packages/arcgis/gis/__init__.py:407: UserWarning: You are logged on as zxuyuan_umich with an administrator role, proceed with caution.
  self.users.me.username)


'CheckedOut'

#### Now you are ready to start!

we first extract all the data to the given directory. To make sure the data has no conflict with the previous operations, we delete all the files that already exists in the directory and start a fresh start~

In [46]:

# give a fresh new start by deleting all of the files that already exists in the folder

def delete_files_except(directory, keep_filenames):
    # Ensure that the path is absolute and normalized
    directory = os.path.abspath(directory)

    # Convert keep_filenames to a set for faster membership testing
    keep_filenames_set = set(keep_filenames)

    # Check if it's a directory, just in case
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist or is not a directory.")
        return

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Construct the full path of the file
            file_path = os.path.join(root, file)
            # Check if the file is not in the list of filenames to keep
            if file not in keep_filenames_set:
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except OSError as e:
                    print(f"Error: {e.strerror}, while trying to delete file {file_path}")

os.chdir('/arcgis/home')
delete_files_except('/arcgis/home', ['fl.zip', 'gr.zip'])

#unzip fl.zip using zipfile
with zipfile.ZipFile(r'fl.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
with zipfile.ZipFile(r'gr.zip', 'r') as zip_ref:
    zip_ref.extractall()

Error: Device or resource busy, while trying to delete file /arcgis/home/fl/.nfs00000000d49f014f00000001
Deleted: /arcgis/home/fl/bg2010.dbf
Deleted: /arcgis/home/fl/bg2010.prj
Deleted: /arcgis/home/fl/bg2010.shp
Deleted: /arcgis/home/fl/bg2010.shx
Deleted: /arcgis/home/fl/flpeop.dbf
Deleted: /arcgis/home/fl/flpeop.prj
Deleted: /arcgis/home/fl/flpeop.shp
Deleted: /arcgis/home/fl/flpeop.shx
Deleted: /arcgis/home/fl/foodpts.cpg
Deleted: /arcgis/home/fl/foodpts.dbf
Deleted: /arcgis/home/fl/foodpts.prj
Deleted: /arcgis/home/fl/foodpts.shp
Deleted: /arcgis/home/fl/foodpts.shp.xml
Deleted: /arcgis/home/fl/foodpts.shx
Deleted: /arcgis/home/fl/output/fas.shp
Deleted: /arcgis/home/fl/output/fas.shx
Deleted: /arcgis/home/fl/output/fas.dbf
Deleted: /arcgis/home/fl/output/fas.cpg
Deleted: /arcgis/home/fl/output/fas.prj
Deleted: /arcgis/home/fl/output/fas.shp.xml
Deleted: /arcgis/home/fl/output/fas.sbn
Deleted: /arcgis/home/fl/output/fas.sbx
Deleted: /arcgis/home/gr/bg2010.dbf
Deleted: /arcgis/home

In [47]:
# List fields using the ListFields function
fields = arcpy.ListFields('/arcgis/home/fl/foodpts.shp')

# Iterate through the fields and print their names
for field in fields:
    print(field.name)

FID
Shape
OBJECTID
ORIGINAL_I
COMPANYNAM
ADDRESS
CITY
STATE
ZIPCODE
CENSUSBLOC
CENSUSTRAC
PRIMARYSIC
PRIMARY_DE
SUB_TYPE
LATITUDE
LONGITUDE
FRANCHISED
EBT_SNAP
DATASET
Shape_1
Date_Revie
fas
oth
ram
eth
far
frm
gro
spc
spm


In [48]:
# List fields using the ListFields function
fields = arcpy.ListFields('/arcgis/home/fl/flpeop.shp')

# Iterate through the fields and print their names
for field in fields:
    print(field.name)

FID
Shape
Id


In [51]:
# you may change the types here
search_radii = [805, 1609, 4828, 8047]  # in meters
food_outlet_types = ['fas', 'oth', 'ram', 'eth', 
                     'far', 'frm', 'gro', 'spc', 'spm']  # Placeholder types


In [52]:
output_directory = '/arcgis/home/fl/output'  # Define an output directory for the new shapefiles

if not os.path.exists(output_directory):
    os.mkdir(output_directory)

for field_name in food_outlet_types:
    # Make a feature layer from the shapefile
    foodpts = arcpy.MakeFeatureLayer_management(shapefile, "foodpts_Layer")

    # Construct a query that selects only features where the value of the field is equal to 1
    query = f"{field_name} = 1"
    arcpy.SelectLayerByAttribute_management(foodpts, "NEW_SELECTION", query)
    
    # Get the number of selected features
    count_result = arcpy.GetCount_management(foodpts)
    count = int(count_result.getOutput(0))
    print(field_name, "has", count, "features selected.")

    if count > 0:
        # Define the path for the new shapefile
        output_shapefile = os.path.join(output_directory, f"{field_name}.shp")
        
        # Copy the selected features to a new shapefile
        arcpy.CopyFeatures_management(foodpts, output_shapefile)
        print(f"Selected features for {field_name} have been saved to {output_shapefile}")

    # Clean up the foodpts layer before the next iteration to prevent schema lock errors
    arcpy.Delete_management(foodpts)

fas has 941 features selected.
Selected features for fas have been saved to /arcgis/home/fl/output/fas.shp
oth has 281 features selected.
Selected features for oth have been saved to /arcgis/home/fl/output/oth.shp
ram has 363 features selected.
Selected features for ram have been saved to /arcgis/home/fl/output/ram.shp
eth has 273 features selected.
Selected features for eth have been saved to /arcgis/home/fl/output/eth.shp
far has 122 features selected.
Selected features for far have been saved to /arcgis/home/fl/output/far.shp
frm has 85 features selected.
Selected features for frm have been saved to /arcgis/home/fl/output/frm.shp
gro has 139 features selected.
Selected features for gro have been saved to /arcgis/home/fl/output/gro.shp
spc has 27 features selected.
Selected features for spc have been saved to /arcgis/home/fl/output/spc.shp
spm has 98 features selected.
Selected features for spm have been saved to /arcgis/home/fl/output/spm.shp


In [ ]:
for outlet_type in food_outlet_types:
    # Apply a selection or somehow isolate each outlet type
    query = f"OutletType = '{outlet_type}'"
    arcpy.SelectLayerByAttribute_management(foodpts, "NEW_SELECTION", query)
    # Kernel Density analysis for each search radius
    for radius in search_radii:
        # Output raster will be named as {outlet_type}_density_{radius}.tif
        out_raster_name = f"{outlet_type}_density_{radius}.tif"
        out_raster_path = arcpy.sa.KernelDensity(foodpts, "NONE", radius)
        # Save the output raster
        out_raster_path.save(out_raster_name)
        print(f"Kernel Density Raster saved: {out_raster_name}")
    # Clear the selection for the next iteration
    arcpy.SelectLayerByAttribute_management(foodpts, "CLEAR_SELECTION")
# Assuming you have a point shapefile of the 1000 people's locations called 'people_locations.shp'
people_locations = "people_locations.shp"

# Sample the kernel density rasters at each people location
for radius in search_radii:
    # We will assume all types have the same search radius in their names
    rasters_to_sample = [arcpy.Raster(f"{outlet_type}_density_{radius}.tif") for outlet_type in food_outlet_types]
    # The Sample tool will take multiple rasters and combine their values at specified locations
    out_table = f"sampled_values_{radius}.dbf"
    arcpy.sa.Sample(rasters_to_sample, people_locations, out_table)
    print(f"Sampled values table saved: {out_table}")

# Check in the Spatial Analyst extension now that we're done
arcpy.CheckInExtension("Spatial")